In [13]:
import pandas as pd
import numpy as np
import scipy.io
import math
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from collections import Counter
from matplotlib.colors import Normalize
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
warnings.filterwarnings("ignore")
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings("ignore")
%matplotlib inline


In [3]:
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):

            pretty(value, indent + 2)
            print('-----------------------')
        else:
            print('\t' * (indent + 2) + str(value))
            print(" ")
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
def rms(arr):
    n = len(arr)
    squared = np.array(arr) * np.array(arr)
    sum = np.sum(squared)
    mean = (sum / (float)(n))
    root = math.sqrt(mean)
    return root
def mav(arr):
    n = len(arr)
    mav = sum((abs(np.array(arr))))/(float)(n)
    return mav
def var(arr):
    n = len(arr)
    squared = np.array(arr) * np.array(arr)
    sum = np.sum(squared)
    result = (sum / (float)(n))
    return result
def wl(arr):
    n = len(arr)
    sum = 0
    for i in range(1, n):
        sum += abs(arr[i] - arr[i - 1])
    return sum
def iav(arr):
    return sum(abs(np.array(arr)))
def extractSubject(name):
    ex1Path = 'DB1/' + name + '/' + name + '_A1_E1.mat'
    print(ex1Path)
    ex1 = scipy.io.loadmat(ex1Path)
    emg = ex1['emg']
    EMGdf = pd.DataFrame.from_dict(emg)
    stimulus = ex1['stimulus']

    ex2Path = 'DB1/' + name + '/' + name + '_A1_E2.mat'
    ex2 = scipy.io.loadmat(ex2Path)
    emg2 = ex2['emg']
    EMGdf2 = pd.DataFrame.from_dict(emg2)
    stimulus2 = ex2['stimulus']

    ex3Path = 'DB1/' + name + '/' + name + '_A1_E3.mat'
    ex3 = scipy.io.loadmat(ex3Path)
    emg3 = ex3['emg']
    EMGdf3 = pd.DataFrame.from_dict(emg3)
    stimulus3 = ex3['stimulus']

    Movements = {}
    for m in range(1, 51):
        if (m < 11):
            movementIndices = np.where(stimulus == m)[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf
        elif (m < 28):
            movementIndices = np.where(stimulus2 == (m - 10))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf2
        else:
            movementIndices = np.where(stimulus3 == (m - 27))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf3

        Electrodes = {}
        for e in range(1, 11):
            temp = {}
            for r in range(1, 7):
                startIndex = repetitions[r - 1][0]
                LastIndex = repetitions[r - 1][len(repetitions[r - 1]) - 1]
                df = EMG.iloc[startIndex:LastIndex, e - 1]
                df.reset_index(drop=True, inplace=True)
                narray = df.to_numpy(dtype=None, copy=False)
                temp["R{0}".format(r)] = narray
            Electrodes["Electrode{0}".format(e)] = temp
        Movements["Movement{0}".format(m)] = Electrodes
    return Movements
def Average(lst):
    return sum(lst) / len(lst)


In [78]:
subjects_accuracy = pd.DataFrame(columns={'Accuracy', 'Accuracy_Modified'})
pca_window = []
pca_movement = []
final_df = pd.DataFrame(columns={'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
                           'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
                           'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
                           'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
                           'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
                           'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
                           'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
                           'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
                           'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
                           'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10',
                           'Train','Movement'})

for s in range(1,28):
    subject = 'S' + str(s)
    dff = pd.DataFrame.from_dict(extractSubject(subject))
    df = pd.DataFrame(columns={'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
                                  'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
                                  'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
                                  'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
                                  'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
                                  'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
                                  'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
                                  'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
                                  'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
                                  'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10',
                                  'Train','Movement'})
    for e in range(1, 11):
        i = 0
        electrode = 'Electrode' + str(e)
        for m in range(1,51):
                M = dff['Movement'+str(m)][electrode]
                for r in range(1, 7):
                    rep = "R" + str(r)
                    if (r in [1, 3, 4, 6]):
                        train = 1
                    else:
                        train = 0
                    print(i)    
                    for x in range(0, len(M[rep]), 16):
                        df.at[i, 'RMS' + str(e)] = rms(M[rep][x:x + 50])
                        df.at[i, 'MAV' + str(e)] = mav(M[rep][x:x + 50])
                        df.at[i, 'VAR' + str(e)] = var(M[rep][x:x + 50])
                        df.at[i, 'WL' + str(e)] = wl(M[rep][x:x + 50])
                        df.at[i, 'IAV' + str(e)] = iav(M[rep][x:x + 50])
                        df.at[i, 'Movement'] = m
                        df.at[i, 'Train'] = train
                    
                        i += 1
                    print(i)    


    final_df = final_df.append(df, ignore_index=True)

DB1/S1/S1_A1_E1.mat
0
33
33
66
66
99
99
131
131
164
164
196
196
229
229
262
262
295
295
328
328
361
361
394
394
427
427
460
460
493
493
526
526
559
559
592
592
625
625
658
658
691
691
724
724
757
757
790
790
823
823
856
856
889
889
922
922
955
955
988
988
1021
1021
1054
1054
1087
1087
1120
1120
1153
1153
1186
1186
1219
1219
1252
1252
1285
1285
1318
1318
1351
1351
1384
1384
1417
1417
1450
1450
1483
1483
1516
1516
1549
1549
1582
1582
1614
1614
1646
1646
1679
1679
1712
1712
1745
1745
1777
1777
1809
1809
1841
1841
1873
1873
1905
1905
1937
1937
1969
1969
2002
2002
2035
2035
2068
2068
2101
2101
2134
2134
2167
2167
2200
2200
2233
2233
2266
2266
2299
2299
2332
2332
2365
2365
2398
2398
2431
2431
2464
2464
2497
2497
2530
2530
2563
2563
2596
2596
2629
2629
2662
2662
2695
2695
2728
2728
2761
2761
2794
2794
2827
2827
2860
2860
2893
2893
2926
2926
2959
2959
2992
2992
3025
3025
3058
3058
3091
3091
3124
3124
3157
3157
3190
3190
3223
3223
3256
3256
3289
3289
3322
3322
3355
3355
3388
3388
3421
3421
3454

In [81]:
features = { 
    'RMS1','RMS2','RMS3','RMS3','RMS4','RMS5','RMS6','RMS7','RMS8','RMS9','RMS10'
            # 'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
            # 'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
            # 'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
            # 'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
            # 'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
            # 'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
            # 'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
            # 'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
            # 'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
            # 'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10'
            }

X_train = final_df[final_df['Train'] == 1].loc[:, features]
X_test = final_df[final_df['Train'] == 0].loc[:, features]
y_train = final_df[final_df['Train'] == 1]['Movement'].astype('int')
y_test = final_df[final_df['Train'] == 0]['Movement'].astype('int')


In [82]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
y_test_new = [most_frequent(y_test[x:x + 33]) for x in range(0, len(y_test), 33)]
y_predicted_new = [most_frequent(y_pred[x:x + 33]) for x in range(0, len(y_pred), 33)]
accuracy_modified = accuracy_score(y_test_new, y_predicted_new)

print("Window Accuracy",accuracy)
print("Movement Accuracy", accuracy_modified)

Window Accuracy 0.4911596668957979
Movement Accuracy 0.8884758364312267
